In [ ]:
! pip install azure-search-documents --pre
! pip install python-dotenv

In [1]:
# Import required libraries  
import os  
import json  
import openai
import keys
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)  
  
# Configure environment variables  
load_dotenv()  
service_endpoint = "https://rsaf-cognitive-search-service-us.search.windows.net"
index_name = "rsaf-cognitive-search"
key = keys.cognitive_key
openai.api_type = "azure"
openai.api_key = "0e576c031e5446e19cc6d866ad3d9f3f"
openai.api_base = "https://raid-ses-openai.openai.azure.com/"
openai.api_version = "2023-05-15"
credential = AzureKeyCredential(key)

In [2]:
client_index = SearchIndexClient(endpoint=service_endpoint, credential=credential)

In [34]:
client_index.delete_index("chat-demo")

In [35]:
for i in client_index.list_indexes():
    print(i.name)

chat-demo


In [36]:
search_client = SearchClient(endpoint=service_endpoint, index_name="chat-demo", credential=credential)

all_docs = search_client.search(search_text="*", filter="page_label eq 5", top=5)

import pandas as pd

pd.DataFrame(all_docs)

,content,page_label,li_jsonMetadata,embedding,li_doc_id,id,@search.score,@search.reranker_score,@search.highlights,@search.captions
0,OFFICIAL (CLOSED) \nA \nOFFICIAL (CLOSED) ...,5,"{""page_label"": ""5"", ""file_name"": ""EEM FINAL ca...","[-0.01596086, -0.0031881887, -0.01879963, -0.0...",f7e3e0f0-697c-418c-9fd2-574119b63c84,7a1a2929-1831-43a3-9ada-733bb89813d7,1.0,None,None,None


In [12]:
search_client = SearchClient(endpoint=service_endpoint, index_name="chat-demo", credential=credential)

In [7]:
results = search_client.search(search_text="Advanced Transitions confined spaces", select="content", top=3)

In [53]:
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="swiftfaq-ada002")
    embeddings = response['data'][0]['embedding']
    return embeddings


In [55]:
# Semantic Hybrid Search
query = "what is autorotation?"

index_name = "chat-demo"

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    query_type="semantic", query_language="en-us", semantic_configuration_name='my-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

HttpResponseError: (FeatureNotSupportedInService) Semantic search is not enabled for this service.
Parameter name: queryType
Code: FeatureNotSupportedInService
Message: Semantic search is not enabled for this service.
Parameter name: queryType
Exception Details:	(SemanticQueriesNotAvailable) Semantic search is not enabled for this service.
	Code: SemanticQueriesNotAvailable
	Message: Semantic search is not enabled for this service.